In [25]:
# Import Required Libraries

import pandas as pd
import math
import numpy as np

import matplotlib.pyplot as plt

import plotly.graph_objs as go
from plotly.offline import iplot

from prophet import Prophet
import holidays
from prophet.diagnostics import cross_validation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
import os

from sklearn.preprocessing import MinMaxScaler
import joblib

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

In [26]:
def is_weekend(ds):
    date = pd.to_datetime(ds)
    # Return True for Saturday (5) and Sunday (6), False otherwise
    return date.weekday() >= 5

def df_to_X_y(df, window_size=6):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np) - window_size):
        row = [r for r in df_as_np[i:i + window_size]]
        X.append(row)
        label = df_as_np[i + window_size][6]  # 'Entry' is the 7th column (index 6)
        y.append(label)
    return np.array(X), np.array(y)

def evaluate_model(test, test_forecast):
    # Evaluate performance
    mse = mean_squared_error(y_true=test['y'],  y_pred=test_forecast['yhat'])
    rmse = np.sqrt(mean_squared_error(y_true=test['y'], y_pred=test_forecast['yhat']))
    mae = mean_absolute_error(y_true=test['y'], y_pred=test_forecast['yhat'])
    r2 = r2_score(y_true=test['y'], y_pred=test_forecast['yhat'])
    return mse, rmse, mae, r2

def evaluatel(y_test_inv, y_pred_inv):
        # Calculate evaluation metrics
    mse = mean_squared_error(y_test_inv, y_pred_inv)
    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r2 = r2_score(y_test_inv, y_pred_inv)
    return mse, rmse, mae, r2

def prophet_model(train, test, df):
    # Set evaluation metrics to 0
    p_rmse = 0
    p_mse = 0
    p_mae = 0
    p_r2 = 0


    # Create holidays dataframe
    holiday = pd.DataFrame([])
    for date, name in sorted(holidays.Philippines(years=[2022, 2023]).items()):
        holiday = pd.concat([holiday, pd.DataFrame({'ds': date, 'holiday': name}, index=[0])], ignore_index=True)
    holiday['ds'] = pd.to_datetime(holiday['ds'], format='%Y-%m-%d', errors='ignore')

    # Initialize the Prophet model
    m = Prophet(
        yearly_seasonality=True,
        daily_seasonality=False,
        weekly_seasonality=True,
        holidays=holiday,
        seasonality_mode='multiplicative',
        scaling='minmax'
    )

    # Add the regressors
    m.add_seasonality(name='daily_is_weekend', period=1, fourier_order=4, condition_name='weekend')
    m.add_seasonality(name='daily_is_weekday', period=1, fourier_order=4, condition_name='weekday')
    m.add_regressor('off_hour')
    m.add_regressor('rain_amount')

    # Fit the model on the training data
    m.fit(train)

    # Make predictions for Test set
    test_forecast = m.predict(test)

    # Remove negative forecasts
    test_forecast['yhat'] = test_forecast['yhat'].apply(lambda x: max(x, 0))
    test_forecast['yhat_lower'] = test_forecast['yhat_lower'].apply(lambda x: max(x, 0))
    test_forecast['yhat_upper'] = test_forecast['yhat_upper'].apply(lambda x: max(x, 0))

    # Round forecast values
    test_forecast['yhat'] = test_forecast['yhat'].round()

    # Create a future DataFrame with hourly intervals for the desired forecast period
    future = m.make_future_dataframe(periods=150, freq='D')
    future['hour'] = pd.to_datetime(future['ds']).dt.hour
    future['is_weekend'] = df['is_weekend']
    future['weekday'] = future['ds'].apply(is_weekend)
    future['weekend'] = ~future['ds'].apply(is_weekend)
    future['rain_amount'] = df['rain_amount']
    future['off_hour'] = future['hour'].apply(lambda x: 1 if (x >= 23) or (x <= 3) else 0)
    future = future[future['ds'].dt.hour < 23]
    future = future[future['ds'].dt.hour > 3]

    # Make predictions for future set
    forecast = m.predict(future)
    
    return test_forecast

def lstm_model(df, X_train, y_train, X_test, y_test, X_val, y_val):
    model = Sequential()
    model.add(InputLayer((X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(64, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='linear'))
    model.add(Dense(1, activation='linear'))
    model.summary()

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Define the ModelCheckpoint callback with the correct file path
    os.makedirs('model', exist_ok=True)
    cp1 = ModelCheckpoint(filepath='model/best_lstm.keras', save_best_only=True, monitor='val_loss', mode='min')

    # Use early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Fit the model
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[cp1, early_stopping])

    # Assuming df is the original DataFrame with the 'Date Time' column
    # Extract the 'Date Time' column for the entire dataset
    date_time_test = df['Date Time']

    # Load the scaler for inverse transformation
    scaler_entry = joblib.load('model/scaler_entry.pkl')

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Inverse transform the predictions and actual values
    # Create a DataFrame to hold the predictions and actual values
    df_pred = pd.DataFrame(y_pred, columns=['Entry'])
    df_actual = pd.DataFrame(y_test, columns=['Entry'])

    # Inverse transform the 'Entry' column
    y_pred_inv = scaler_entry.inverse_transform(df_pred)
    y_test_inv = scaler_entry.inverse_transform(df_actual)

    # Ensure date_time_test matches the length of y_test_inv and y_pred_inv
    date_time_test = date_time_test[-len(y_test_inv):]

    # Round the predictions to the nearest whole number and ensure non-negative values
    y_pred_inv = np.round(np.maximum(y_pred_inv, 0))
    y_test_inv = np.round(np.maximum(y_test_inv, 0))

    # Calculate evaluation metrics
    l_mse = mean_squared_error(y_test_inv, y_pred_inv)
    l_rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    l_mae = mean_absolute_error(y_test_inv, y_pred_inv)
    l_r2 = r2_score(y_test_inv, y_pred_inv)

    return y_pred_inv



In [27]:
# Import dataset
df = pd.read_csv('data/00_MRT_2023_Prophet_weather2.csv', parse_dates=[0])

# Rename header to Prophet's requirements
df.reset_index()
df = df.rename(columns={'Datetime':'ds', 'Entry':'y'})

# Add additional regressors as columns in the dataframe
df['hour'] = pd.to_datetime(df['ds']).dt.hour
df['off_hour'] = df['hour'].apply(lambda x: 1 if (x >= 23) or (x <= 3) else 0)
df['weekday'] = ~df['ds'].apply(is_weekend)
df['weekend'] = df['ds'].apply(is_weekend)

# Split the dataset to training and testing sets
train_len = math.floor((df.shape[0]*80)/100)
train = df[:train_len]
test = df[train_len:]

test_forecast = prophet_model(train, test, df)
prophet_eval = evaluate_model(test, test_forecast)
print(prophet_eval)


C:\Users\inspiron-7460\AppData\Local\Temp\ipykernel_16800\3373363910.py:2: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\inspiron-7460\AppData\Local\Temp\ipykernel_16800\410437130.py:45: FutureWarning:

errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead

18:47:56 - cmdstanpy - INFO - Chain [1] start processing
18:48:25 - cmdstanpy - INFO - Chain [1] done processing


(388428.54452054796, 623.2403585460011, 424.76849315068495, 0.802910465276687)


In [28]:
# Load your time series data
df2 = pd.read_csv('data/2223TaftLSTM.csv')

# Set if holiday
ph_holidays = holidays.PH()
df2['is_holiday'] = df2['Date'].apply(lambda x: 1 if x in ph_holidays else 0)
df2.head(5)

# Combine 'Date' and 'Time' into a new column 'DateAndTime'
df2['DateAndTime'] = pd.to_datetime(df2['Date'] + ' ' + df2['Time'])

# Drop unneeded columns
df2.drop(['Date', 'Time', 'rain_amount', 'rain_desc'], axis=1, inplace=True)

 # Define additional features
df2['Date Time'] = pd.to_datetime(df2['DateAndTime'], format='%d.%m.%Y %H.%M.%S')
df2['hour'] = df2['Date Time'].dt.hour
df2['day_of_week'] = df2['Date Time'].dt.dayofweek
df2['month'] = df2['Date Time'].dt.month
df2['year'] = df2['Date Time'].dt.year
df2.drop(['DateAndTime', 'Day', 'is_weekend'], axis=1, inplace=True)
df2['is_weekend'] = df2['day_of_week'].apply(lambda x: 1 if x in [5, 6] else 0)
model_features = ['hour', 'day_of_week', 'is_weekend', 'month', 'year', 'rain_class', 'Entry']
df_model = df2[model_features]

# Normalize all features except 'Entry'
scaler = MinMaxScaler(feature_range=(0, 1))
df_model[df_model.columns[:-1]] = scaler.fit_transform(df_model[df_model.columns[:-1]])

# Save the scaler for the features
joblib.dump(scaler, 'model/scaler_features.pkl')

# Normalize the 'Entry' column separately
scaler_entry = MinMaxScaler(feature_range=(0, 1))
df_model['Entry'] = scaler_entry.fit_transform(df_model[['Entry']])

# Save the scaler for the 'Entry' column
joblib.dump(scaler_entry, 'model/scaler_entry.pkl')

#
X, y = df_to_X_y(df_model)

# Determine the split points
train_split_point = int(len(X) * 0.8)
valntest_split_point = int(len(X) * 0.9)

# Split the data
X_train, X_val, X_test = X[:train_split_point], X[train_split_point:valntest_split_point], X[valntest_split_point:]
y_train, y_val, y_test = y[:train_split_point], y[train_split_point:valntest_split_point], y[valntest_split_point:]

lstm_forecast = lstm_model(df2, X_train, y_train, X_test, y_test, X_val, y_val)

C:\Users\inspiron-7460\AppData\Local\Temp\ipykernel_16800\3982592567.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\inspiron-7460\AppData\Local\Temp\ipykernel_16800\3982592567.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 6, 64)          │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,569 (209.25 KB)

 Trainable params: 53,569 (209.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
365/365 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.0344 - val_loss: 0.0086
Epoch 2/100
365/365 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0088 - val_loss: 0.0056
Epoch 3/100
365/365 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0061 - val_loss: 0.0052
Epoch 4/100
365/365 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0052 - val_loss: 0.0046
Epoch 5/100
365/365 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0048 - val_loss: 0.0041
Epoch 6/100
365/365 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0040 - val_loss: 0.0036
Epoch 7/100
365/365 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: 0.0037 - val_loss: 0.0032
Epoch 8/100
365/365 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 0.0034 - val_loss: 0.0025
Epoch 9/100
365/365 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0029 - val_loss: 0.0022
Epoch 10/100
365/365 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0027 - val_loss: 0.0024
Epoch 11/100
365/365 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 12/100
365/365 ━━━━━━━━

In [29]:
lstm_forecast = pd.DataFrame(lstm_forecast, columns=['LSTM'])
lstm_forecast


,LSTM
0,105.0
1,2428.0
2,4576.0
3,5264.0
4,4590.0
...,...
1455,2020.0
1456,1524.0
1457,309.0
1458,55.0


In [30]:
test_forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Additional special (non-working) day,Additional special (non-working) day_lower,Additional special (non-working) day_upper,All Saints' Day,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2023-08-08 04:00:00,1497.136465,0.000000,485.085061,1497.136465,1497.136465,0.0,0.0,0.0,0.0,...,0.089071,0.089071,0.089071,-0.034198,-0.034198,-0.034198,0.0,0.0,0.0,0.0
1,2023-08-08 05:00:00,1497.195138,2156.484798,3311.776344,1497.195138,1497.195138,0.0,0.0,0.0,0.0,...,0.085238,0.085238,0.085238,-0.033700,-0.033700,-0.033700,0.0,0.0,0.0,2749.0
2,2023-08-08 06:00:00,1497.253810,3852.449866,5031.972010,1497.253810,1497.253810,0.0,0.0,0.0,0.0,...,0.081228,0.081228,0.081228,-0.033200,-0.033200,-0.033200,0.0,0.0,0.0,4458.0
3,2023-08-08 07:00:00,1497.312483,4208.409712,5442.564886,1497.312483,1497.312483,0.0,0.0,0.0,0.0,...,0.077100,0.077100,0.077100,-0.032698,-0.032698,-0.032698,0.0,0.0,0.0,4825.0
4,2023-08-08 08:00:00,1497.371156,3755.499387,4898.769066,1497.371156,1497.371156,0.0,0.0,0.0,0.0,...,0.072909,0.072909,0.072909,-0.032192,-0.032192,-0.032192,0.0,0.0,0.0,4303.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,2023-12-31 19:00:00,1702.197674,0.000000,8533.188138,-606.817374,4104.932264,0.0,0.0,0.0,0.0,...,-0.077551,-0.077551,-0.077551,-0.031309,-0.031309,-0.031309,0.0,0.0,0.0,3540.0
2916,2023-12-31 20:00:00,1702.256347,0.000000,7091.576600,-607.321001,4106.656773,0.0,0.0,0.0,0.0,...,-0.067652,-0.067652,-0.067652,-0.030808,-0.030808,-0.030808,0.0,0.0,0.0,2996.0
2917,2023-12-31 21:00:00,1702.315019,0.000000,4639.539584,-607.824627,4108.381283,0.0,0.0,0.0,0.0,...,-0.057386,-0.057386,-0.057386,-0.030307,-0.030307,-0.030307,0.0,0.0,0.0,1906.0
2918,2023-12-31 22:00:00,1702.373692,0.000000,840.780349,-608.328254,4110.105793,0.0,0.0,0.0,0.0,...,-0.046840,-0.046840,-0.046840,-0.029808,-0.029808,-0.029808,0.0,0.0,0.0,190.0


In [31]:
# Get the number of rows
num_rows = len(test_forecast)

# Calculate the midpoint
midpoint = num_rows // 2

# Take the latter half
prophet_forecast = test_forecast[midpoint:]
hybrid_actual = test[midpoint:]

# Reset the index of the DataFrame and drop the old index
prophet_forecast.reset_index(drop=True, inplace=True)
hybrid_actual.reset_index(drop=True, inplace=True)

print(prophet_forecast)

                      ds        trend   yhat_lower   yhat_upper  trend_lower  \
0    2023-10-20 04:00:00  1599.931097     0.000000   228.079678   886.949609   
1    2023-10-20 05:00:00  1599.989769  1382.313874  4057.672894   886.435185   
2    2023-10-20 06:00:00  1600.048442  2463.788274  6661.661203   885.935419   
3    2023-10-20 07:00:00  1600.107115  2520.082040  7312.252499   885.520837   
4    2023-10-20 08:00:00  1600.165788  2293.664146  6460.716359   885.106255   
...                  ...          ...          ...          ...          ...   
1455 2023-12-31 19:00:00  1702.197674     0.000000  8533.188138  -606.817374   
1456 2023-12-31 20:00:00  1702.256347     0.000000  7091.576600  -607.321001   
1457 2023-12-31 21:00:00  1702.315019     0.000000  4639.539584  -607.824627   
1458 2023-12-31 22:00:00  1702.373692     0.000000   840.780349  -608.328254   
1459 2023-12-31 23:00:00  1702.432365     0.000000   804.740385  -608.831880   

      trend_upper  Additional special (

In [32]:
hybrid_forecast = prophet_forecast[['ds', 'yhat']].rename(columns={'yhat': 'prophet'})
hybrid_forecast['lstm'] = lstm_forecast['LSTM']
hybrid_forecast['y'] = hybrid_actual['y']
print(hybrid_forecast)

                      ds  prophet    lstm     y
0    2023-10-20 04:00:00      0.0   105.0     0
1    2023-10-20 05:00:00   2694.0  2428.0  2894
2    2023-10-20 06:00:00   4534.0  4576.0  4339
3    2023-10-20 07:00:00   4940.0  5264.0  5548
4    2023-10-20 08:00:00   4396.0  4590.0  4502
...                  ...      ...     ...   ...
1455 2023-12-31 19:00:00   3540.0  2020.0  1681
1456 2023-12-31 20:00:00   2996.0  1524.0   297
1457 2023-12-31 21:00:00   1906.0   309.0     0
1458 2023-12-31 22:00:00    190.0    55.0     0
1459 2023-12-31 23:00:00    201.0   110.0     0

[1460 rows x 4 columns]


In [41]:
# Define weights
weight_prophet = 0.9
weight_lstm = 0.1
# Calculate the hybrid forecast
hybrid_forecast['yhat'] = (
    weight_prophet * hybrid_forecast['prophet'] + 
    weight_lstm * hybrid_forecast['lstm']
)

actual = hybrid_forecast['y']
forecast = hybrid_forecast['yhat']

#Evaluate hybrid forecast
mse, rmse, mae, r2 = evaluate_model(hybrid_forecast, hybrid_forecast)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

MSE: 377843.6935
RMSE: 614.6899
MAE: 429.5564
R²: 0.7942
